# Data Cleaning

This notebook is used to clean and combine all datasources into a coherent dataset. 

We have three different data groups:
* Label & Patient Data
* Lab Data
* MRI Data

All of these have to be cleaned, selected and combined into one dataset.

## Imports

In [1]:
import pandas as pd
import datetime
import numpy as np
import re

# Data Cleaning and Selection Label & Patient-data
First we will clean the data for our labels and patients. This includes the basic cleaning and data type matching as well as looking at anomalies and define the output format. We will use the prepared 'no duplicate PID' Sheet with Labels from KSA.

In [2]:
print("Start Clean and Preprocessing patients-data")

Start Clean and Preprocessing patients-data


In [3]:
df_patients = pd.read_excel(r'../raw_data/Hypophysenpatienten.xlsx',sheet_name='no duplicate PID')
df_patients.head()

,%ID,Fall Nr.,Datum/Zeit,Modalität,Exam Code,Exam Name,Abteilung,Arbeitsplatz.Kürzel,Aufnahmeart,PID,...,OP Datum,Ausfälle post,Diagnose,Kategorie,Patient Alter,Zuweiser,AnforderungDatum,ÜberweiserIntern.Bereich,ÜberweiserIntern.Klinik,Gender
0,8548799,41324884,2021-12-21 14:58:00,M-MR,MNDSHYP,MR Hypophyse,MRN,MRI4,Amb,4858,...,NaT,NaN,NaN,NaN,76,Endo,2021-11-09 14:38:55.0000000,MUK,Endokrinologie,male
1,7786984,40619017,2019-06-12 10:16:00,M-MR,MNDSHYP,MR Hypophyse,MRN,MRI4,Amb,6144,...,NaT,NaN,NaN,NaN,63,Neurochir,2018-09-04 15:03:12.0000000,KCH,Neurochirurgie,male
2,8899566,41814658,2023-03-07 10:29:00,M-MR,MNDSHYP,MR Hypophyse,MRN,MRI2,Amb,7048,...,NaT,NaN,NaN,NaN,60,Augenklinik,2023-01-25 19:43:52.0000000,KCH,Augenklinik,male
3,8376664,41172058,2021-03-27 15:05:00,M-MR,MNDSHYP,MR Hypophyse,MRN,MRI4,Amb,7375,...,NaT,NaN,inaktiv,non-prolaktinom,55,Endo,2021-03-22 18:29:03.0000000,MUK,Endokrinologie,male
4,7430454,40279590,2017-09-12 11:05:00,M-MR,MNDSHYP,MR Hypophyse,MRN,MRI3,Amb,14433,...,NaT,keine,inaktiv,non-prolaktinom,77,Berkmann Sven,2016-10-18 14:12:19.0000000,KCH,Neurochirurgie,female


In [4]:
df_patients.tail()

,%ID,Fall Nr.,Datum/Zeit,Modalität,Exam Code,Exam Name,Abteilung,Arbeitsplatz.Kürzel,Aufnahmeart,PID,...,OP Datum,Ausfälle post,Diagnose,Kategorie,Patient Alter,Zuweiser,AnforderungDatum,ÜberweiserIntern.Bereich,ÜberweiserIntern.Klinik,Gender
521,8711158,41598956,2022-08-16 13:13:00,M-MR,MNDSHYP,MR Hypophyse,MRN,MRI3,Amb,300375656,...,2022-05-23,keine,inaktiv,non-prolaktinom,68,Hirntumorzen,2022-06-02 16:05:45.0000000,KCH,Hirntumorzentrum,female
522,8686428,41596179,2022-07-12 12:15:00,M-MR,MNDSHYP,MR Hypophyse,MRN,MRI2,Amb,300375717,...,NaT,NaN,NaN,NaN,69,Rudofsky Gottfried,2022-05-04 08:17:31.0000000,-,-,male
523,8760431,41664583,2022-10-27 11:54:00,M-MR,MNDSHYP,MR Hypophyse,MRN,MRI1,Amb,300387138,...,NaT,keine,prolaktinom,prolaktinom,72,Neurochir,2022-08-05 09:51:39.0000000,KCH,Neurochirurgie,male
524,8884823,41792671,2023-01-09 18:12:00,M-MR,MNDSHYP,MR Hypophyse,MRN,MRI3,Stat,300394756,...,NaT,NaN,NaN,NaN,56,421 Neurochirurgische Station,2023-01-09 17:37:29.0000000,-,-,male
525,8251904,40950091,2020-10-14 08:13:00,M-MR,MNDSHYP,MR Hypophyse,MRN,MRI3,Amb,580000175,...,NaT,NaN,NaN,NaN,33,Endo,2020-09-23 17:21:06.0000000,MUK,Endokrinologie,female


## Basic Cleaning, Column Selection, Anomaly Correction and Format definition


### Column Selection
First we select only the columns which have value to our model or our analysis. Some columns are already renamed to make their content more intuitive.

In [5]:
# define needed columns
column_list = ['PID','Fall Nr.',"Datum/Zeit","Arbeitsplatz.Kürzel",'Grösse',
       'Ausfälle prä', 'Qualität', 'ED','OP Datum',
       'Diagnose', 'Kategorie', 'Patient Alter',
       'Prolaktin',"IGF1", 'Cortisol','fT4','weiteres Labor','Gender']
df_patients = df_patients[column_list]
# rename columns
df_patients= df_patients.rename(columns={"Fall Nr.": "Case_ID","PID": "Patient_ID",
                       "Datum/Zeit": "Date_Case","ED": "Entry_date", "OP Datum": "Operation_date",
                       "Arbeitsplatz.Kürzel":"ID_MRI_Machine","Grösse": "Adenoma_size","Qualität": "Label_Quality",
                       "Patient Alter":"Patient_age","Kategorie":"Category","Diagnose":"Diagnosis",
                       "Prolaktin":"Prolactin","weiteres Labor":"Lab_additional", 'Gender':"Patient_gender"})

### Check for Anomalies and correct them
There are some Anomalies mostly in the datetime columns (eg. Operation date before Entry Date). These are corrected or were corrected by the KSA after feedback from us. 

In [6]:
# rows where Entry Date is after Operationdate?
assert len(df_patients[df_patients['Operation_date'] < df_patients['Entry_date']][['Entry_date','Operation_date']]) ==0

### Data Type Definition
Now we check the column data-types and parse them into their resprective type if not already correct.


In [7]:
# make datetime values
df_patients["Date_Case"] = pd.to_datetime(df_patients["Date_Case"])
df_patients["Entry_date"] = pd.to_datetime(df_patients["Entry_date"])
df_patients["Operation_date"] = pd.to_datetime(df_patients["Operation_date"])

In [8]:
# set category data type in pandas, check datatypes
df_patients['ID_MRI_Machine'] = df_patients['ID_MRI_Machine'].astype('category')
df_patients['Adenoma_size'] = df_patients['Adenoma_size'].astype('category')
df_patients['Diagnosis'] = df_patients['Diagnosis'].astype('category')
df_patients['Category'] = df_patients['Category'].astype('category')
df_patients['Patient_gender'] = df_patients['Patient_gender'].astype('category')
df_patients.dtypes

Patient_ID                 int64
Case_ID                    int64
Date_Case         datetime64[ns]
ID_MRI_Machine          category
Adenoma_size            category
Ausfälle prä              object
Label_Quality             object
Entry_date        datetime64[ns]
Operation_date    datetime64[ns]
Diagnosis               category
Category                category
Patient_age                int64
Prolactin                 object
IGF1                      object
Cortisol                  object
fT4                       object
Lab_additional            object
Patient_gender          category
dtype: object

### Check Duplicates
Check if a patient is duplicated.

In [9]:
# Patient ID Duplicate Check
assert len(df_patients[df_patients["Patient_ID"].duplicated()]) == 0

## One Hot Encode Categorical Values

To use and analyse the categorical data we need to one-hot encode them. This is done by splitting the comma separated strings into single strings and then create a one-hot-encoded column of each individual value. This column is then added to the original dataframe.

In [10]:
df_patients["Ausfälle prä"]= df_patients["Ausfälle prä"].str.replace(' ', '')
df_patients["Ausfälle prä"]= df_patients["Ausfälle prä"].str.lower()
# Split the 'Ausfälle prä' column into separate strings
df_patients['Ausfälle prä'] = df_patients['Ausfälle prä'].str.split(',')

# Create a set to store all unique disfunctions
unique_disfunctions = set()

# Iterate over the 'Ausfälle prä' column to gather unique disfunctions
for value in df_patients['Ausfälle prä']:
    if isinstance(value, list):
        unique_disfunctions.update(value)
    elif isinstance(value, str):
        unique_disfunctions.add(value)

# Iterate over the unique disfunctions and create one-hot encoded columns
for disfunction in unique_disfunctions:
    df_patients["Pre_OP_hormone_"+ disfunction] = df_patients['Ausfälle prä'].apply(lambda x: 1 if (isinstance(x, list) and disfunction in x) or (x == disfunction) else 0)
# drop the original 'Ausfälle prä' column
df_patients = df_patients.drop('Ausfälle prä', axis=1)

## Remove All NA and not needed Labels

In [11]:
# remove all labels which are not prolaktion or non-prolaktinom
df_patients = df_patients[df_patients['Category'].isin(['non-prolaktinom','prolaktinom'])]
assert len(df_patients['Category'].unique()) == 2

In [12]:
df_patients.to_csv(r'../raw_data/label_data.csv',index=False)

In [13]:
print("End Clean and Preprocessing patient data")

End Clean and Preprocessing patient data


# Data Cleaning and Selection MRI-data

Now we will clean all MRI's.

In [14]:
print("Start Clean and Preprocessing mri data")

Start Clean and Preprocessing mri data


### Column Selection
Only select the interesting columns for the mri's.

In [15]:
column_list_mri = ['PID','Fall Nr.',"Datum/Zeit","Arbeitsplatz.Kürzel",'%ID']

In [16]:
df_mri = pd.read_excel(r'../raw_data/Hypophysenpatienten.xlsx',sheet_name='w duplicates')
# select and rename columns
df_mri = df_mri[column_list_mri]
df_mri= df_mri.rename(columns={"Fall Nr.": "Case_ID","PID": "Patient_ID",
                       "Datum/Zeit": "Date_Case","Arbeitsplatz.Kürzel":"ID_MRI_Machine",'%ID':"MRI_Case_ID",})

In [17]:
df_mri.head()

,Patient_ID,Case_ID,Date_Case,ID_MRI_Machine,MRI_Case_ID
0,300146159,0041835743,2023-05-11 09:00:00,MRI3,8921949
1,762512,0041708812,2023-05-06 08:12:00,MRI3,8947649
2,365189,0041892695,2023-05-05 14:19:00,MRI3,8688141
3,543641,0041725372,2023-05-05 07:54:00,MRI2,8930353
4,300302329,0041843364,2023-05-02 12:04:00,MRI3,8930425


In [18]:
df_mri.tail()

,Patient_ID,Case_ID,Date_Case,ID_MRI_Machine,MRI_Case_ID
1193,112374,0004213315,2016-01-19 07:45:00,MRI3,7279605
1194,153807,0004211936,2016-01-12 11:40:00,MRI2,7272498
1195,719666,0004180070,2016-01-07 11:45:00,MRI2,7247437
1196,313269,0004139115,2016-01-05 07:00:00,MRI3,7272286
1197,637049,0004210941,2016-01-04 12:30:00,MRI1,7271507


In [19]:
df_mri['Case_ID'] = df_mri['Case_ID'].replace('-', '', regex=True)

In [20]:
df_mri['Case_ID'] = df_mri['Case_ID'].astype(int)

In [21]:
# #TODO: Get MRI shortly before Operation
df_mri_merge = df_mri.merge(df_patients[['Patient_ID','Operation_date']], how='left', on="Patient_ID")
df_mri_op = df_mri_merge[~df_mri_merge['Operation_date'].isna()]
df_mri_nonop = df_mri_merge[df_mri_merge['Operation_date'].isna()]
df_mri_op= df_mri_op[(df_mri_op['Date_Case'] <= df_mri_op['Operation_date']) ]
df_mri = pd.concat([df_mri_op,df_mri_nonop]).reset_index(drop=True)

## Remove same Day MRI and take only newest

In [22]:
n_cases = len(df_mri)
df_mri_clean = df_mri.groupby(["Patient_ID","Case_ID"])[["Date_Case",'ID_MRI_Machine',"MRI_Case_ID"]].max().reset_index()

# if there are multiple
df_mri_clean = df_mri_clean.groupby(["Patient_ID","Case_ID"])[["Date_Case",'ID_MRI_Machine',"MRI_Case_ID"]].max().reset_index()
print(f"{n_cases-len(df_mri_clean)} Cases were deleted, because they were same-day duplicates.")

26 Cases were deleted, because they were same-day duplicates.


In [23]:
df_mri_clean.to_csv(r'../raw_data/mri_data.csv',index=False)

In [24]:
print("End Clean and Preprocessing mri data")

End Clean and Preprocessing mri data


# Data Cleaning and Selection Lab-data

Now the lab data from the explicit KSA export will be cleaned.

In [25]:
print("Start Clean and Preprocessing lab-data")

Start Clean and Preprocessing lab-data


### Read

In [26]:
lab_data = pd.read_excel("../raw_data/extract_pit.xlsx",
                         usecols=['PATIENT_NR','FALL_NR','Analyse-ID','Resultat','Datum_Resultat']).rename(
                             columns={"PATIENT_NR":"Patient_ID","FALL_NR":"Case_ID","Analyse-ID":"Lab_ID",})

In [27]:
lab_data.columns

Index(['Case_ID', 'Patient_ID', 'Lab_ID', 'Datum_Resultat', 'Resultat'], dtype='object')

### Clean and select Lab's
There is a multitude of labs in the export. We do not need all of them. 

In [28]:
# remove not needed labs
lab_data= lab_data[~lab_data['Lab_ID'].isin(['ABTEST','TBILHB'])].copy()

In [29]:
# rename labs which are integer based with a string name
lab_data['Lab_ID'] = lab_data['Lab_ID'].replace({20396:'IGF1',24382:'PROL',24384:'PROL',24383:'PROL',
                                                 'COR30':'COR','COR60':'COR',
                                                 'CORL0':'COR','CORL30':'COR',
                                                 'COR0':'COR'})

In [30]:
# replace some not used characters in the case ids
lab_data['Case_ID'] = lab_data['Case_ID'].replace('#','',regex=True)
lab_data['Case_ID'] = lab_data['Case_ID'].astype(int)

In [31]:
# clean result column
lab_data['Resultat'] = lab_data['Resultat'].replace(',','.',regex=True)
lab_data['Resultat'] = lab_data['Resultat'].replace('>','',regex=True)
lab_data['Resultat'] = lab_data['Resultat'].replace('<','',regex=True)
lab_data['Resultat'] = lab_data['Resultat'].replace('¬†','',regex=True)
lab_data['Resultat']= lab_data['Resultat'].astype(float)

In [32]:
# replace export anomalies 
ids = {'Ã¼': 'ü', 'Ã¤': 'ä', "Ã„":"Ä","√§":"ä"}

for column in lab_data.columns[lab_data.columns.isin(["Case_ID","Patient_ID","Datum_Resultat","Auftragsdatum"]) == False]:
    for old, new in ids.items():
        lab_data[column] = lab_data[column].replace(old, new, regex=False)

# clean the greather and less than characters with regex
clean_result = lambda result: re.sub(r'(?<!\d)\.', '', re.sub(r'[^\d.]', '', str(result))) #clean < zahl / > zahl / 1 A zahl
lab_data["Resultat"] = lab_data["Resultat"].apply(clean_result) 
# remove empty results
lab_data = lab_data[lab_data["Resultat"] != ""]
lab_data["Resultat"] = lab_data["Resultat"].astype(float)

In [33]:
# check if the datetime was correctly fixed
assert lab_data["Datum_Resultat"].min() > pd.to_datetime("1995-01-01")

In [34]:
# mean of results of same date
lab_data = lab_data.groupby(["Patient_ID","Lab_ID","Datum_Resultat"])["Resultat"].agg(['mean']).reset_index()

### Merge Cases with Patient Cases

In [35]:
lab_data = pd.merge(lab_data,df_mri_clean.loc[:,["Patient_ID","Case_ID","Date_Case"]],on="Patient_ID",how = "right")
lab_data = lab_data[lab_data["Date_Case"] >= lab_data["Datum_Resultat"]].drop(columns="Date_Case")

In [36]:
# Compute newest date for each patient and analysis
max_dates = lab_data.groupby(['Patient_ID', "Lab_ID","Case_ID"])['Datum_Resultat'].max().reset_index()
# Merge with the original DataFrame to filter rows with minimum dates
lab_data = pd.merge(lab_data, max_dates, on=['Patient_ID', 'Lab_ID', 'Datum_Resultat',"Case_ID"])

In [37]:
# check for any duplicate Values
assert len(lab_data.loc[:,["Case_ID","Lab_ID"]].drop_duplicates()) == len(lab_data)

In [38]:
# make dataframe wide
lab_data = lab_data.pivot(index=["Patient_ID","Case_ID"],values = ['mean'], columns = ['Lab_ID'])
lab_data.columns = lab_data.columns.droplevel()
lab_data = lab_data.reset_index()

### Create LabData from label data

In [39]:
df_additional_lab = pd.read_csv(r'../raw_data/label_data.csv').rename(columns={'Cortisol':'COR','fT4':'FT4','Prolactin':'PROL'})[['Patient_ID','Case_ID','COR','FT4','PROL','IGF1','Lab_additional']]
df_additional_lab = df_additional_lab.dropna(subset=['PROL','IGF1','COR','FT4','Lab_additional'], how='all').reset_index(drop=True)

In [40]:
df_additional_lab['Lab_additional'] = df_additional_lab['Lab_additional'].fillna('')
for i in ['Test', 'LH','FSH']:
    df_additional_lab[i] = ''
    indices = df_additional_lab[df_additional_lab['Lab_additional'].str.contains(i)].index
    df_additional_lab.loc[indices,i] = df_additional_lab.iloc[indices]['Lab_additional']

In [41]:
df_additional_lab = df_additional_lab.drop(columns=['Lab_additional'])
df_additional_lab = df_additional_lab.rename(columns={'Test':'TEST'})

#### Testosteron Cleaning

In [42]:
df_additional_lab['TEST'] = df_additional_lab['TEST'].replace(r' nmol/l','',regex=True)
df_additional_lab['TEST'] = df_additional_lab['TEST'].replace(r'nmol/l','',regex=True)
df_additional_lab['TEST'] = df_additional_lab['TEST'].replace(r'nmol','',regex=True)
df_additional_lab['TEST'] = df_additional_lab['TEST'].replace(r'Testo','',regex=True)
df_additional_lab['TEST'] = df_additional_lab['TEST'].replace(r'Test','',regex=True)
df_additional_lab['TEST'] = df_additional_lab['TEST'].replace(r',','.',regex=True)
df_additional_lab.loc[df_additional_lab['TEST'] == '', 'TEST'] = np.nan
df_additional_lab['TEST']= df_additional_lab['TEST'].astype(float)

#### LH Cleaning

In [43]:
df_additional_lab.loc[df_additional_lab['LH'] == '', 'LH'] = np.nan
df_additional_lab['LH'] = df_additional_lab['LH'].replace(r'FSH \d*U\/L,','',regex=True)
df_additional_lab['LH'] = df_additional_lab['LH'].replace(r'LH','',regex=True)
df_additional_lab['LH'] = df_additional_lab['LH'].replace(r'U/L','',regex=True)
df_additional_lab['LH']= df_additional_lab['LH'].astype(float)

#### FSH Cleaning

In [44]:
df_additional_lab['FSH'] = df_additional_lab['FSH'].replace(r'FSH','',regex=True)
df_additional_lab['FSH'] = df_additional_lab['FSH'].replace(r'U/L','',regex=True)
df_additional_lab['FSH'] = df_additional_lab['FSH'].replace(r', LH \d*','',regex=True)

df_additional_lab.loc[df_additional_lab['FSH'] == '', 'FSH'] = np.nan
df_additional_lab['FSH']= df_additional_lab['FSH'].astype(float)

#### Cortisol Cleaning 

In [45]:
df_additional_lab['COR'] = df_additional_lab['COR'].replace(r' nmol/l','',regex=True)
df_additional_lab['COR']= df_additional_lab['COR'].astype(float)

#### FT4 Cleaning

In [46]:
df_additional_lab['FT4'] = df_additional_lab['FT4'].replace(r' pmol/l','',regex=True)
df_additional_lab['FT4']= df_additional_lab['FT4'].astype(float)

#### Prolaktin Cleaning and Conversion

In [47]:
df_additional_lab["PROL"]= df_additional_lab["PROL"].str.replace("ug/L","ug/l")
df_additional_lab["PROL"]= df_additional_lab["PROL"].str.replace("mu/L","mU/l")


In [48]:
# get indices which need to be converted
# indices_to_divide = df_additional_lab.loc[df_additional_lab["PROL"].str.contains('mU/l'),'PROL'].index 
indices_to_divide = df_additional_lab[~df_additional_lab["PROL"].isna() & df_additional_lab['PROL'].str.contains('mU/l')].index 
# remove units and strings
df_additional_lab['PROL'] = df_additional_lab['PROL'].str.rstrip(r'mU/l')
df_additional_lab['PROL'] = df_additional_lab['PROL'].str.rstrip(r'ug/l')
df_additional_lab['PROL'] = df_additional_lab['PROL'].str.rstrip(r'ug/L')
df_additional_lab['PROL'] = df_additional_lab['PROL'].astype(float)
# mU/l -> ug/l (mU/l * 0.048)
df_additional_lab.loc[indices_to_divide,'PROL'] = df_additional_lab.loc[indices_to_divide,'PROL'] * 0.048 


#### IGF1 Cleaning and Conversion

In [49]:
# df_additional_lab["IGF1"]= df_additional_lab["IGF1"].str.replace("ug/L","ug/l")
df_additional_lab["IGF1"]= df_additional_lab["IGF1"].str.replace("ug/l","")
df_additional_lab["IGF1"]= df_additional_lab["IGF1"].str.replace(",",".")

# get indices which need to be converted
# indices_to_divide = df_additional_lab.loc[df_additional_lab["IGF1"].str.contains('ng/ml'),'IGF1'].index 
indices_to_divide = df_additional_lab[~df_additional_lab["IGF1"].isna() & df_additional_lab['IGF1'].str.contains('ng/ml')].index
# remove units and strings
df_additional_lab['IGF1'] = df_additional_lab['IGF1'].str.rstrip(r'ng/ml')
df_additional_lab['IGF1'] = df_additional_lab['IGF1'].str.rstrip(r'nmol')
df_additional_lab['IGF1'] = df_additional_lab['IGF1'].str.rstrip(r'nmol/l')
df_additional_lab['IGF1'] = df_additional_lab['IGF1'].astype(float)
# ng/ml -> nmol/l (ng/ml * 0.13)
df_additional_lab.loc[indices_to_divide,'IGF1'] = df_additional_lab.loc[indices_to_divide,'IGF1'] * 0.13


In [50]:
# combine additional lab and lab data
lab_data = lab_data.set_index(['Patient_ID','Case_ID']).combine_first(df_additional_lab.set_index(['Patient_ID','Case_ID'])).reset_index()

In [51]:
spar = round(lab_data[['COR', 'FSH','LH', 'FT4', 'IGF1', 'LH', 'PROL','TEST']].isna().mean().mean(),3)
print(f"Sparsity of labordata: {spar} % (nur von Fällen mit Laborwerten)")
print(f"Von {len(df_mri_clean)-len(lab_data)} Fällen gibt es keine Laborwerte.")

Sparsity of labordata: 0.26 % (nur von Fällen mit Laborwerten)
Von 299 Fällen gibt es keine Laborwerte.


In [52]:
lab_data.to_csv(r'../raw_data/lab_data.csv',index=False)

In [53]:
print("End Clean and Preprocessing labor data")

End Clean and Preprocessing labor data


# Full Merge

In [54]:
# get mridata where an mri matches to with case and patient if of a lab
df_temp = pd.merge(df_mri_clean,lab_data,left_on=['Patient_ID','Case_ID'],right_on=['Patient_ID','Case_ID'])

In [55]:
# match this data with the patients data
full_merged = pd.merge(df_temp,df_patients[['Patient_ID','Category','Patient_gender','Patient_age','Adenoma_size']+
                                           list(df_patients.columns[df_patients.columns.str.contains('Pre')])+
                                           list(df_patients.columns[df_patients.columns.str.contains('Diagnosis_')])+
                                              ['Operation_date', 'Entry_date']]
                                              ,how='inner',left_on=['Patient_ID'],right_on=['Patient_ID'])

In [56]:
full_merged = full_merged.sort_values('Patient_ID')

### Delete Cases

There are no MRI-files for these Cases.

In [57]:
no_mri_cases = [41040108, 40632245]
full_merged = full_merged[full_merged["Case_ID"].isin(no_mri_cases)==False]

In [58]:
assert full_merged.duplicated(subset=['Patient_ID','Case_ID']).sum() == 0

In [59]:
full_merged.to_csv(r'../raw_data/data_full_merge.csv',index=False)